In [3]:
import pandas as pd

In [1]:
def load_dataset_ddi(path):
    asymmetric_ddi = pd.read_csv(path, delimiter=",")
    asymmetric_ddi.rename(columns={'EffectorDrugID': 'precipitantDrug', 'AffectedDrugID': 'objectDrug',
                                   'EffectorDrugLabel': 'precipitantDrugLabel', 'AffectedDrugLabel': 'objectDrugLabel'},
                          inplace=True)
    asymmetric_ddi.loc[asymmetric_ddi['Effect'].isin(
        ['Excretion_rate', 'Excretory_function', 'excretion_rate', 'excretion rate']), 'Effect'] = 'excretion'
    asymmetric_ddi.loc[asymmetric_ddi['Effect'].isin(['Process_of_absorption', 'Absorption']),
                       'Effect'] = 'absorption'
    asymmetric_ddi.loc[asymmetric_ddi['Effect'].isin(
        ['Serum_concentration', 'Serum_concentration_of', 'Serum_level', 'serum level', 'serum_concentration',
         'serum concentration', 'serum concentration of ']), 'Effect'] = 'serum_concentration'
    asymmetric_ddi.loc[asymmetric_ddi['Effect'].isin(['Metabolism', 'metabolism']), 'Effect'] = 'metabolism'

    asymmetric_ddi = asymmetric_ddi.loc[
        asymmetric_ddi.Effect.isin(['excretion', 'absorption', 'serum_concentration', 'metabolism'])]

#     asymmetric_ddi = asymmetric_ddi[
#         ['precipitantDrugLabel', 'objectDrugLabel', 'Effect', 'Impact', 'precipitantDrug', 'objectDrug']]

    asymmetric_ddi.loc[asymmetric_ddi.Impact.isin(
        ['Increase', 'Higher', 'Worsening', 'higher', 'increase', 'worsening']), 'Impact'] = 'increase'
    asymmetric_ddi.loc[asymmetric_ddi.Impact.isin(['Decrease', 'Lower', 'Reduced', 'Reduction']),
                       'Impact'] = 'decrease'

#     asymmetric_ddi = asymmetric_ddi.dropna()
    asymmetric_ddi.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    return asymmetric_ddi

def get_ddi_triples(pk_ddi):
    set_drugs = set()
    graph_ttl = """@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
    @prefix ex: <http://example/#> .
    @prefix dpi: <http://example/DrugProteinInteraction#> .
    @prefix ddi: <http://example/DrugDrugInteraction#> ."""
    save_ttl_file(graph_ttl, 'PK_DDI_DPI.ttl')
    for index, row in pk_ddi.iterrows():
        graph_ttl = ''
        if row['precipitantDrug'] not in set_drugs:
            graph_ttl = graph_ttl + """<http://example/Drug/""" + row['precipitantDrug'] + """>\trdf:type\tex:Drug .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['precipitantDrug'] + """>\trdf:hasName\t""" + row['precipitantDrugLabel'] + """ .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['precipitantDrug'] + """>\trdf:hasID\t""" + row['precipitantDrug'] + """ .\n"""
            set_drugs.add(row['precipitantDrug'])
        if row['objectDrug'] not in set_drugs:
            graph_ttl = graph_ttl + """<http://example/Drug/""" + row['objectDrug'] + """>\trdf:type\tex:Drug .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['objectDrug'] + """>\trdf:hasName\t""" + row['objectDrugLabel'] + """ .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['objectDrug'] + """>\trdf:hasID\t""" + row['objectDrug'] + """ .\n"""
            set_drugs.add(row['objectDrug'])
        
        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\trdf:type\tex:DDI .\n"""
        
        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\tex:Effect\tex:""" + row['Effect'] + """ .\n"""
        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\tex:Impact\tex:""" + row['Impact'] + """ .\n"""

        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\tex:precipitant_drug\t<http://example/Drug/""" + row['precipitantDrug'] + """> .\n"""
        graph_ttl = graph_ttl + """ddi:""" + row['precipitantDrug'] + row[
            'objectDrug'] + row['Effect'] + """\tex:object_drug\t<http://example/Drug/""" + row['objectDrug'] + """> .\n"""
        save_ttl_file(graph_ttl, 'PK_DDI_DPI.ttl')
    return set_drugs

def get_dpi_triples(dpi, set_drugs):
    set_protein = set()
    for index, row in dpi.iterrows():
        #organism = row['organism'].replace(' ', '_')
        graph_ttl = ''
        if row['drugbank_id'] not in set_drugs:
            graph_ttl = graph_ttl + """<http://example/Drug/""" + row['drugbank_id'] + """>\trdf:type\tex:Drug .\n"""
            #graph_ttl = graph_ttl + """<http://example/Drug/""" + row['drugbank_id'] + """>\trdf:hasID\t""" + row['drugbank_id'] + """ .\n"""
            set_drugs.add(row['drugbank_id'])
        if row['uniprot_id'] not in set_protein:
            graph_ttl = graph_ttl + """<http://example/Protein/""" + row['uniprot_id'] + """>\trdf:type\tex:Protein .\n"""
            set_protein.add(row['uniprot_id'])
        
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] +  """\trdf:type\tex:DPI .\n"""
        
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] +  """\tex:hasProtein\t<http://example/Protein/""" + row['uniprot_id'] + """> .\n"""
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] +  """\tex:hasDrug\t<http://example/Drug/""" + row['drugbank_id'] + """> .\n"""
        
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] + """\tex:hasCategory\tex:""" + row['category'] + """ .\n"""
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] + """\tex:hasGeneID\tex:""" + str(row['entrez_gene_id']) + """ .\n"""
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] + '\tex:hasOrganism\t"' + row['organism'] + '"@en .\n'
        graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
            'uniprot_id'] + '\tex:hasKnownAction\t"' + row['known_action'] + '"@en .\n'
        if type(row['pubmed_ids'])!=float:
            graph_ttl = graph_ttl + """dpi:""" + row['drugbank_id'] + row[
                'uniprot_id'] + '\tex:hasPubmedID\t"' + str(row['pubmed_ids']) + '"@en .\n'
        save_ttl_file(graph_ttl, 'PK_DDI_DPI.ttl')
    return set_drugs, set_protein

def save_ttl_file(graph_ttl, name):
    with open(name, 'a') as file:
        file.write(graph_ttl)

In [47]:
asymmetric_ddi.Effect.unique()

array(['anticoagulant activities', 'therapeutic efficacy',
       'antiplatelet activities', 'serum concentration',
       'thrombogenic activities', 'hypotensive activities',
       'orthostatic hypotensive activities ', 'metabolism',
       'immunosuppressive activities', 'hyperkalemic activities',
       'hypertensive activities', 'nephrotoxic activities', 'absorption',
       'adverse effects', 'efficacy', 'excretion', 'excretion rate',
       'serum level', 'myopathic rhabdomyolysis activities',
       'hypolipidaemic activities', 'QTc-prolonging activities',
       'hepatotoxic activities',
       'central nervous system depressant (CNS depressant) activities',
       'hypokalemic activities', 'bradycardic activities',
       'arrhythmogenic activities', 'bioavailability',
       'cardiotoxic activities',
       'atrioventricular blocking (AV block) activities',
       'serotonergic activities', 'neurotoxic activities',
       'vasopressor activities', 'orthostatic hypotensive ac

In [4]:
path = 'Data/'
pk_ddi = load_dataset_ddi(path+'PharmacokineticDDI.csv')
dpi = pd.read_csv(path+'DrugsAndProteins.csv', delimiter=",")
display(pk_ddi.shape, pk_ddi.head(2))
display(dpi.shape, dpi.head(2))

C:\Users\rivasa\AppData\Local\Temp/ipykernel_1668/3541643745.py:2: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  pk_ddi = load_dataset_ddi(path+'PharmacokineticDDI.csv')


(617815, 10)

,Interaction,precipitantDrug,objectDrug,precipitantDrugLabel,objectDrugLabel,Effect,Adverse_Event,Impact,EffectorDrugCUI,AffectedDrugCUI
0,The serum concentration of Alteplase can be de...,DB00727,DB00009,Nitroglycerin,Alteplase,serum_concentration,NaN,decrease,C0017887,C0032143
1,The metabolism of Cyclosporine can be decrease...,DB06605,DB00091,Apixaban,Cyclosporine,metabolism,NaN,decrease,C1831808,C0010592


(19906, 8)

,drugbank_id,category,uniprot_id,entrez_gene_id,organism,known_action,actions,pubmed_ids
0,DB00001,target,P00734,2147,Human,yes,inhibitor,10505536|10912644|11055889|11467439|11807012|1...
1,DB00006,target,P00734,2147,Human,yes,inhibitor,11060732|11504570|11833835|11923794|11929334|1...


In [5]:
pk_ddi.to_csv('PharmacokineticDDI.csv', index=None)

In [37]:
asymmetric_ddi = pd.read_csv(path+'PharmacokineticDDI.csv', delimiter=",")
asymmetric_ddi.head(2)

C:\Users\rivasa\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Interaction,EffectorDrugID,AffectedDrugID,EffectorDrugLabel,AffectedDrugLabel,Effect,Adverse_Event,Impact,EffectorDrugCUI,AffectedDrugCUI
0,Apixaban may increase the anticoagulant activi...,DB06605,DB00001,Apixaban,Lepirudin,anticoagulant activities,NaN,increase,C1831808,C0772394
1,Apixaban may increase the anticoagulant activi...,DB06605,DB00001,Apixaban,Lepirudin,anticoagulant activities,NaN,increase,C1831808,C0772394


In [10]:
df1 = asymmetric_ddi[['EffectorDrugID', 'EffectorDrugLabel', 'EffectorDrugCUI']]
df1.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df2 = asymmetric_ddi[['AffectedDrugID', 'AffectedDrugLabel', 'AffectedDrugCUI']]
df2.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df1.rename(columns={'EffectorDrugID': 'DrugID', 'EffectorDrugLabel': 'DrugName',
                                   'EffectorDrugCUI': 'DrugCUI'}, inplace=True)
df2.rename(columns={'AffectedDrugID': 'DrugID', 'AffectedDrugLabel': 'DrugName',
                                   'AffectedDrugCUI': 'DrugCUI'}, inplace=True)

df1 = pd.concat([df1, df2])
df1.drop_duplicates(keep='first', inplace=True, ignore_index=True)
df1.to_csv('Drugs.csv', index=None)
df1

C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
C:\Users\rivasa\Anaconda3\lib\site-packages\pandas\core\frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,DrugID,DrugName,DrugCUI
0,DB06605,Apixaban,C1831808
1,DB06695,Dabigatran etexilate,C1571583
2,DB09070,Tibolone,C0076660
3,DB00013,Urokinase,C0042071
4,DB00163,Vitamin E,C0042874
...,...,...,...
3501,DB11148,Butamben,C0054235
3502,DB11502,Butacaine,C0006466
3503,DB13328,Butanilicaine,C0054241
3504,DB13578,Metabutethamine,C0301289


In [25]:
df2 = dpi[['pubmed_ids']]
dpi_pubmed = pd.DataFrame()
for index, row in dpi.iterrows():
    if type(row['pubmed_ids'])!=float:
        df3 = pd.DataFrame(data=row['pubmed_ids'].split('|'), columns=['pubmed_ids'])
        df3['drugbank_id'] = row['drugbank_id']
        df3['uniprot_id'] = row['uniprot_id']
        dpi_pubmed = pd.concat([dpi_pubmed, df3])
    
dpi_pubmed

,pubmed_ids,drugbank_id,uniprot_id
0,10505536,DB00001,P00734
1,10912644,DB00001,P00734
2,11055889,DB00001,P00734
3,11467439,DB00001,P00734
4,11807012,DB00001,P00734
...,...,...,...
0,17591945,DB08889,P40306
0,23090647,DB08890,P25092
0,23590413,DB08907,P13866
0,23056222,DB08908,Q14145


In [26]:
dpi_pubmed.to_csv('dpi_pubmed.csv', index=None)

In [4]:
set_drugs = get_ddi_triples(pk_ddi)
set_drugs, set_protein = get_dpi_triples(dpi, set_drugs)

In [5]:
from collections import OrderedDict

In [6]:
def save_ttl_file(graph_ttl):
    #open text file
    ttl_file = open("PK_DDI_DPI_00.ttl", "w")
    #write string to file
    n = ttl_file.write(graph_ttl)
    #close file
    ttl_file.close()

In [7]:
graph_ttl = open("PK_DDI_DPI.ttl").read()
graph_ttl = "\n".join(list(OrderedDict.fromkeys(graph_ttl.split("\n"))))
save_ttl_file(graph_ttl)

In [9]:
from SPARQLWrapper import SPARQLWrapper, JSON
from rdflib.plugins.sparql.processor import SPARQLResult

In [10]:
dbpedia = SPARQLWrapper('https://dbpedia.org/sparql/')

In [48]:
drug = set(pk_ddi.precipitantDrug.unique())
d = set(pk_ddi.objectDrug.unique())
drug.update(d)
d = set(dpi.drugbank_id.unique())
drug.update(d)
len(drug)

6693

In [79]:

name = """','""".join(drug)
name = "'"+name+"'"


In [82]:
df1

,db_id,relation,dbpedia_resource
0,DB03974,dbo:drugbank,NaN
1,DB00503,dbo:drugbank,http://dbpedia.org/resource/Ritonavir
2,DB13264,dbo:drugbank,NaN
3,DB00588,dbo:drugbank,http://dbpedia.org/resource/Fluticasone
4,DB07829,dbo:drugbank,NaN
...,...,...,...
6688,DB00733,dbo:drugbank,NaN
6689,DB02272,dbo:drugbank,NaN
6690,DB02028,dbo:drugbank,NaN
6691,DB09185,dbo:drugbank,NaN


In [84]:
df1 = pd.DataFrame(data=drug, columns=['db_id'])
df1['relation'] = 'dbo:drugbank'

for index, row in df1.iterrows():
    query = """select distinct ?s where {
?s dbo:drugbank '""" + row['db_id'] + """' .}"""
    dbpedia.setQuery(query)
    dbpedia.setReturnFormat(JSON)
    results = dbpedia.query().convert()
    if len(results['results']['bindings'])>0:
        df1.at[index, 'dbpedia_resource'] = results['results']['bindings'][0]['s']['value']

df1

,db_id,relation,dbpedia_resource
0,DB03974,dbo:drugbank,NaN
1,DB00503,dbo:drugbank,http://dbpedia.org/resource/Ritonavir
2,DB13264,dbo:drugbank,NaN
3,DB00588,dbo:drugbank,http://dbpedia.org/resource/Fluticasone
4,DB07829,dbo:drugbank,NaN
...,...,...,...
6688,DB00733,dbo:drugbank,http://dbpedia.org/resource/Pralidoxime
6689,DB02272,dbo:drugbank,NaN
6690,DB02028,dbo:drugbank,NaN
6691,DB09185,dbo:drugbank,http://dbpedia.org/resource/Viloxazine


In [86]:
len(df1.dbpedia_resource.unique())

1765